In [1]:
import pickle
import glob

fl = sorted(glob.glob('../features/*.pkl'))
print(fl)

['../features/other_feat.pkl', '../features/resnet_feat.pkl', '../features/s_cnn_feat.pkl', '../features/x_feat.pkl']


In [2]:
import numpy as np
first_f = True
train_x, train_y, test_x = [],[],[]
for f in fl:
    with open(f,'rb') as fin:
        a,b,c = pickle.load(fin)
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        
        if first_f is True:
            train_y = b
            first_f = False
            
        train_x.append(a)
        test_x.append(c)
print(len(train_x),len(train_x[0]))

4 43999


In [3]:
all_train_x = np.hstack(train_x)
print(all_train_x.shape)
all_test_x = np.hstack(test_x)
print(all_test_x.shape)
print(train_y.shape)

(43999, 94)
(2640, 94)
(43999,)


In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
def cv_test(k_cnt=3):
    rnd = 420
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    test_pred = None
    for train_index, test_index in kf.split(all_train_x,train_y):
        X_train, X_test = all_train_x[train_index], all_train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        model = XGBClassifier()
        model.fit(X_train,y_train)
        
        val_pred = model.predict(X_test)
        print(accuracy_score(y_test,val_pred))
        if test_pred is None:
            test_pred = model.predict_proba(all_test_x)
        else:
            test_pred += model.predict_proba(all_test_x)
        print('=============================')
    return test_pred
    
print('def done')

def done


In [5]:
import pandas as pd
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

def gen_sub(k=3):
    test_pred = cv_test(k)
    y_res = np.argmax(test_pred,axis=1)
    y_res = [list_classes[i] for i in y_res]
    df = pd.read_csv('../input/sample_submission.csv')
    f_name = [p.split('/')[-1] for p in test_files]
    df['fname'] = f_name
    df['camera'] = y_res
    print(df.head(10))
    df.to_csv('../results/xgb_{}fold.csv'.format(k),index=False)

gen_sub(3)

0.990456714383
0.989978866998
0.98963165075
                   fname             camera
0  img_0002a04_manip.tif        LG-Nexus-5x
1  img_001e31c_unalt.tif          iPhone-4s
2  img_00275cf_manip.tif           iPhone-6
3  img_0034113_unalt.tif  Samsung-Galaxy-S4
4  img_00344b7_unalt.tif         Motorola-X
5  img_0062ce8_manip.tif  Samsung-Galaxy-S4
6  img_00885c3_manip.tif           HTC-1-M7
7  img_008a869_unalt.tif           HTC-1-M7
8  img_008fb06_manip.tif  Samsung-Galaxy-S4
9  img_00c98af_manip.tif        LG-Nexus-5x


In [6]:
gen_sub(5)

0.990909090909
0.988863636364
0.992045454545
0.990113636364
0.988066825776
                   fname             camera
0  img_0002a04_manip.tif        LG-Nexus-5x
1  img_001e31c_unalt.tif          iPhone-4s
2  img_00275cf_manip.tif           iPhone-6
3  img_0034113_unalt.tif  Samsung-Galaxy-S4
4  img_00344b7_unalt.tif         Motorola-X
5  img_0062ce8_manip.tif  Samsung-Galaxy-S4
6  img_00885c3_manip.tif           HTC-1-M7
7  img_008a869_unalt.tif           HTC-1-M7
8  img_008fb06_manip.tif  Samsung-Galaxy-S4
9  img_00c98af_manip.tif        LG-Nexus-5x
